In [1]:
%matplotlib notebook
import csv
from scipy.io import loadmat
import numpy as np
import scipy.optimize as opt
from pandas.io.parsers import read_csv
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from sklearn.preprocessing import PolynomialFeatures

In [2]:
data = loadmat('../ex5data1.mat')
y = data['y']
x = data['X']
yval = data['yval']
xval = data['Xval']
ytest = data['ytest']
xtest = data['Xtest']
th = np.ones(2)

In [3]:
np.dot(x,th[1:]).shape[0]

12

In [20]:
def coste(th,x,y,reg = 1):
    m = x.shape[0]
    print (np.dot(x,th[1:]))
    print(y[:,0])
    print(np.dot(x,th[1:]) - y[:,0])
    return (1/(2*m)) * np.sum(np.square(np.dot(x,th[1:]) - y[:,0])) + (reg/(2*m)) * np.sum(np.square(th[1:])) 

coste(th,x,y)

[-15.93675813 -29.15297922  36.18954863  37.49218733 -48.05882945
  -8.94145794  15.30779289 -34.70626581   1.38915437 -44.38375985
   7.01350208  22.76274892]
[ 2.13431051  1.17325668 34.35910918 36.83795516  2.80896507  2.12107248
 14.71026831  2.61418439  3.74017167  3.73169131  7.62765885 22.7524283 ]
[-1.80710686e+01 -3.03262359e+01  1.83043945e+00  6.54232168e-01
 -5.08677945e+01 -1.10625304e+01  5.97524583e-01 -3.73204502e+01
 -2.35101730e+00 -4.81154512e+01 -6.14156770e-01  1.03206175e-02]


319.79620789446545

In [36]:
def gradiente(th,x,y,reg=1):
    print(np.dot(x, th[1:])-y[:,0])
    t0 = np.sum(np.dot(x, th[1:])-y[:,0] * x[0])/len(x)
    t1 = np.sum(np.dot( (np.dot(x, th[1:])-y[:,0])[1:] ,x[1:]))/len(x) + reg * th[1:]/len(x)
    print(t0.shape, t1.shape)
    return np.insert(t1, 0, t0, axis=0)

In [37]:
coste(th,x,y)

[-15.93675813 -29.15297922  36.18954863  37.49218733 -48.05882945
  -8.94145794  15.30779289 -34.70626581   1.38915437 -44.38375985
   7.01350208  22.76274892]
[ 2.13431051  1.17325668 34.35910918 36.83795516  2.80896507  2.12107248
 14.71026831  2.61418439  3.74017167  3.73169131  7.62765885 22.7524283 ]
[-1.80710686e+01 -3.03262359e+01  1.83043945e+00  6.54232168e-01
 -5.08677945e+01 -1.10625304e+01  5.97524583e-01 -3.73204502e+01
 -2.35101730e+00 -4.81154512e+01 -6.14156770e-01  1.03206175e-02]


319.79620789446545

In [38]:
gradiente(th,x,y)

[-1.80710686e+01 -3.03262359e+01  1.83043945e+00  6.54232168e-01
 -5.08677945e+01 -1.10625304e+01  5.97524583e-01 -3.73204502e+01
 -2.35101730e+00 -4.81154512e+01 -6.14156770e-01  1.03206175e-02]
() (1,)


array([173.68658157, 579.33664968])

In [ ]:
np.square(np.array([[1,2],[3,4]]))